In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

# Especifica la ruta al archivo Excel
archivo_excel = 'extract_glosa.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            934 non-null    int64         
 1   rutemisor     934 non-null    object        
 2   folio         934 non-null    int64         
 3   date          934 non-null    datetime64[ns]
 4   xml_dte       934 non-null    object        
 5   convert_from  934 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 43.9+ KB


In [2]:
# SinDscItem = pd.DataFrame()
# df_con = pd.DataFrame()
filas_sin = []
filas_con = []

i = 0

for xml in df['convert_from'].head(20):

    root = ET.fromstring(xml)

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None: # Crea nuevas filas para el df con DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
            "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
        }
        filas_con.append(nuevaFila)
    else: # Crea nuevas filas para el df sin DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
        }
        filas_sin.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con)
df_sin = pd.DataFrame(filas_sin)

print(f'filas totales: {df_con.shape[0]+df_sin.shape[0]}')

filas totales: 20


In [28]:
def df_transform(df):
    archivo_excel = 'extract_glosa.xlsx'

    # Lee el archivo Excel en un DataFrame de Pandas
    # df = pd.read_excel(archivo_excel)
    filas_sin = []
    filas_con = []

    i = 0

    for xml in df['convert_from'].head(20):

        root = ET.fromstring(xml)

        # Utiliza el namespace de XML
        namespace = {'default': 'http://www.sii.cl/SiiDte'}

        rut_emisor = root.find('.//default:RutEmisor', namespace).text

        razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

        NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

        # a veces DscItem no existe
        DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

        if DscItem is not None: # Crea nuevas filas para el df con DscItem
            nuevaFila = {
                "rut_emisor": rut_emisor,
                "razon_social": razon_social,
                "NmbItem": NmbItem,
                # "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
            }
            filas_con.append(nuevaFila)
        else: # Crea nuevas filas para el df sin DscItem
            nuevaFila = {
                "rut_emisor": rut_emisor,
                "razon_social": razon_social,
                "NmbItem": NmbItem,
            }
            filas_sin.append(nuevaFila)

    # agregar las nuevas filas a los distintos df
    df_con = pd.DataFrame(filas_con)
    df_sin = pd.DataFrame(filas_sin)
    df_new = pd.concat([df_con, df_sin], axis=0, ignore_index=False)
    return df_new
    

In [32]:
def leer_saltar(df, inicio, leer=2, saltar=10):
    # Inicializamos una lista para almacenar los DataFrames seleccionados
    lista_dfs = []
    
    # Recorremos el DataFrame con un paso de leer * 3 (desfasado para tres funciones)
    for i in range(inicio, len(df), leer * 3):
        # Seleccionamos las filas correspondientes
        df_leido = df.iloc[i:i+leer]
        # Añadimos las filas leídas a la lista
        lista_dfs.append(df_leido)
        
        # Imprimimos los índices de las filas leídas
        print(f'Leyendo filas con índices: {list(df_leido.index)}')
    
    # Concatenamos los DataFrames leídos en uno solo
    df_resultado = pd.concat(lista_dfs, ignore_index=True)
    
    return df_resultado

# print("Función 1:")
# df_resultado1 = leer_saltar(df, inicio=0, leer=5)
# print("\nFunción 2:")
# df_resultado2 = leer_saltar(df, inicio=5, leer=5)
# print("\nFunción 3:")
# df_resultado3 = leer_saltar(df, inicio=10, leer=5)

In [47]:
# Modificar la función leer_saltar
def leer_saltar2(df, inicio, leer=2):
    # Inicializamos una lista para almacenar los DataFrames seleccionados
    lista_dfs = []
    
    # Recorremos el DataFrame con un paso de leer * 3 (desfasado para tres funciones)
    for i in range(inicio, len(df), leer * 3):
        # Seleccionamos las filas correspondientes
        df_leido = df.iloc[i:i+leer]
        
        # Aplicamos la transformación df_transform a las filas leídas
        df_transformado = df_transform(df_leido)
        
        # Añadimos las filas transformadas a la lista
        lista_dfs.append(df_transformado)
        
        # Imprimimos los índices de las filas leídas
        print(f'Leyendo y transformando filas con índices: {list(df_leido.index)}')
    
    # Concatenamos los DataFrames transformados en uno solo
    df_resultado = pd.concat(lista_dfs, ignore_index=True)
    
    return df_resultado

In [48]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            934 non-null    int64         
 1   rutemisor     934 non-null    object        
 2   folio         934 non-null    int64         
 3   date          934 non-null    datetime64[ns]
 4   xml_dte       934 non-null    object        
 5   convert_from  934 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 43.9+ KB
None


In [49]:
print("Función 1:")
df_resultado1 = leer_saltar2(df, inicio=0, leer=5)
print("\nFunción 2:")
df_resultado2 = leer_saltar2(df, inicio=5, leer=5)
print("\nFunción 3:")
df_resultado3 = leer_saltar2(df, inicio=10, leer=5)

Función 1:
Leyendo y transformando filas con índices: [0, 1, 2, 3, 4]
Leyendo y transformando filas con índices: [15, 16, 17, 18, 19]
Leyendo y transformando filas con índices: [30, 31, 32, 33, 34]
Leyendo y transformando filas con índices: [45, 46, 47, 48, 49]
Leyendo y transformando filas con índices: [60, 61, 62, 63, 64]
Leyendo y transformando filas con índices: [75, 76, 77, 78, 79]
Leyendo y transformando filas con índices: [90, 91, 92, 93, 94]
Leyendo y transformando filas con índices: [105, 106, 107, 108, 109]
Leyendo y transformando filas con índices: [120, 121, 122, 123, 124]
Leyendo y transformando filas con índices: [135, 136, 137, 138, 139]
Leyendo y transformando filas con índices: [150, 151, 152, 153, 154]
Leyendo y transformando filas con índices: [165, 166, 167, 168, 169]
Leyendo y transformando filas con índices: [180, 181, 182, 183, 184]
Leyendo y transformando filas con índices: [195, 196, 197, 198, 199]
Leyendo y transformando filas con índices: [210, 211, 212, 213,

In [50]:
df_resultado1.head()

,rut_emisor,razon_social,NmbItem
0,77566800-8,INVERSIONES COZFER LTDA,Arriendo local 2 Puerto
1,76789922-K,EDGECONNEX CHILE SPA,Usage: Remote Hands 1 Day
2,16383874-5,LUIS SEBASTIAN SAAVEDRA GUAJARDO,Arriendo Álvaro Casanova
3,96689310-9,TRANSBANK S.A.,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...
4,96689310-9,TRANSBANK S.A.,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...
